In [11]:
import pandas as pd
import numpy as np


In [4]:
df_application_train = pd.read_csv('application_train.csv',header=0,error_bad_lines=False,index_col=0,nrows=5000)
df_previous_application = pd.read_csv('previous_application.csv',header=0,error_bad_lines=False,index_col=0,nrows=5000)
df_installment_payments = pd.read_csv('installments_payments.csv',header=0,error_bad_lines=False,index_col=0,nrows=5000)
df_bureau = pd.read_csv('bureau.csv',header=0,error_bad_lines=False,index_col=0,nrows=5000)
df_bureau_bal = pd.read_csv('bureau_balance.csv',header=0,error_bad_lines=False,index_col=0,nrows=5000)
df_pos_cash_bal = pd.read_csv('POS_CASH_balance.csv',header=0,error_bad_lines=False,index_col=0,nrows=5000)
df_credit_card_bal = pd.read_csv('credit_card_balance.csv',header=0,error_bad_lines=False,nrows=5000)

dflist = [df_application_train,df_previous_application,df_installment_payments,df_bureau,df_bureau_bal,df_pos_cash_bal,df_credit_card_bal]


In [5]:
df = df_application_train.merge(df_previous_application,on =['SK_ID_CURR'],suffixes=('_app_train','_prev_app'))


In [9]:
df_bureau.columns

Index(['SK_ID_BUREAU', 'CREDIT_ACTIVE', 'CREDIT_CURRENCY', 'DAYS_CREDIT',
       'CREDIT_DAY_OVERDUE', 'DAYS_CREDIT_ENDDATE', 'DAYS_ENDDATE_FACT',
       'AMT_CREDIT_MAX_OVERDUE', 'CNT_CREDIT_PROLONG', 'AMT_CREDIT_SUM',
       'AMT_CREDIT_SUM_DEBT', 'AMT_CREDIT_SUM_LIMIT', 'AMT_CREDIT_SUM_OVERDUE',
       'CREDIT_TYPE', 'DAYS_CREDIT_UPDATE', 'AMT_ANNUITY'],
      dtype='object')

In [12]:
df_bureau_pivot = pd.pivot_table(df_bureau,values=df_bureau.columns,index='SK_ID_CURR',aggfunc={'SK_ID_BUREAU':'count','CREDIT_ACTIVE':'count','CREDIT_CURRENCY':(lambda x: len(x.unique())),'DAYS_CREDIT':np.min,'CREDIT_DAY_OVERDUE':'sum','DAYS_CREDIT_ENDDATE':'sum', 'DAYS_ENDDATE_FACT':'sum', 'AMT_CREDIT_MAX_OVERDUE':'sum', 'CNT_CREDIT_PROLONG':'sum', 'AMT_CREDIT_SUM':'sum','AMT_CREDIT_SUM_DEBT':'sum', 'AMT_CREDIT_SUM_LIMIT':'sum', 'AMT_CREDIT_SUM_OVERDUE':'sum',
       'CREDIT_TYPE':(lambda x: len(x.unique())), 'DAYS_CREDIT_UPDATE':np.min, 'AMT_ANNUITY':'sum'})

In [15]:
df_bureau_pivot.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1004 entries, 100568 to 456116
Data columns (total 16 columns):
AMT_ANNUITY               1004 non-null float64
AMT_CREDIT_MAX_OVERDUE    1004 non-null float64
AMT_CREDIT_SUM            1004 non-null float64
AMT_CREDIT_SUM_DEBT       1004 non-null float64
AMT_CREDIT_SUM_LIMIT      1004 non-null float64
AMT_CREDIT_SUM_OVERDUE    1004 non-null float64
CNT_CREDIT_PROLONG        1004 non-null int64
CREDIT_ACTIVE             1004 non-null int64
CREDIT_CURRENCY           1004 non-null int64
CREDIT_DAY_OVERDUE        1004 non-null int64
CREDIT_TYPE               1004 non-null int64
DAYS_CREDIT               1004 non-null int64
DAYS_CREDIT_ENDDATE       1004 non-null float64
DAYS_CREDIT_UPDATE        1004 non-null int64
DAYS_ENDDATE_FACT         1004 non-null float64
SK_ID_BUREAU              1004 non-null int64
dtypes: float64(8), int64(8)
memory usage: 133.3 KB
